In [184]:
import pymysql
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, struct

In [185]:
def mysql_connect():
    return pymysql.connect(host='localhost', port=3306, user='user', password='pass123', db='uol')
sql = 'SELECT * FROM registration'

In [186]:
# Get pdf from pandas
pdf = pd.read_sql(sql, con=mysql_connect())

In [187]:
# Create data frame from pdf with spark
spark = SparkSession.builder.appName('singleUserService').getOrCreate()
df = spark.createDataFrame(pdf)

In [188]:
df.groupBy('registration_id', 'registration_ip').pivot('registration_id').agg({'*': 'count'}).withColumn('count_struct', struct('registration_ip', 'registration_id')).show()

+---------------+---------------+----+----+----+----+------------------+
|registration_id|registration_ip|   1|   2|   3|   4|      count_struct|
+---------------+---------------+----+----+----+----+------------------+
|              1|  187.95.19.235|   1|null|null|null|{187.95.19.235, 1}|
|              2|  187.95.19.235|null|   1|null|null|{187.95.19.235, 2}|
|              3|  187.95.19.235|null|null|   1|null|{187.95.19.235, 3}|
|              4|  187.95.19.235|null|null|null|   1|{187.95.19.235, 4}|
+---------------+---------------+----+----+----+----+------------------+

